In [45]:
import pandas as pd
import altair as alt
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# --- 1. CHARGEMENT DES DONNÉES ---
file_excel = "indicateurs.csv (7).xlsx"
sheet_eboulements = "Eboulements"
sheet_meteo = "Météo"
sheet_mer = "Etat de la mer"
sheet_marnage = "Marnage"

df_eboulements = pd.read_excel(file_excel, sheet_name=sheet_eboulements)
df_meteo = pd.read_excel(file_excel, sheet_name=sheet_meteo)
df_mer = pd.read_excel(file_excel, sheet_name=sheet_mer)
df_marnage = pd.read_excel(file_excel, sheet_name=sheet_marnage)
# Nettoyage des noms de colonnes (espaces, accents, etc.)
def clean_columns(df):
    df.columns = df.columns.str.strip()
    return df

df_eboulements = clean_columns(df_eboulements)
df_meteo = clean_columns(df_meteo)
df_mer = clean_columns(df_mer)
df_marnage = clean_columns(df_marnage)

# Renommer la colonne éboulements si nécessaire
if 'nb_éboulement ' in df_eboulements.columns:
    df_eboulements.rename(columns={'nb_éboulement ': 'nb_éboulement'}, inplace=True)
if 'nb_éboulement' in df_eboulements.columns:
    df_eboulements.rename(columns={'nb_éboulement': 'nb_éboulements'}, inplace=True)

# Nettoyer les périodes
for df in [df_eboulements, df_meteo, df_mer]:
    for col in df.columns:
        if 'period' in col.lower() or 'période' in col.lower():
            df[col] = df[col].astype(str).str.strip()

# --- 2. GESTION DES PÉRIODES ET DURÉES ---
period_map = {
    # Format Long (Météo / Mer),
    '2001-01-01 → 2008-12-31': ('2001-2008', 8),
    '2009-01-01 → 2012-12-31': ('2009-2012', 4),
    '2013-01-01 → 2015-12-31': ('2013-2015', 3),
    '2016-01-01 → 2019-12-31': ('2016-2019', 4),
    '2020-01-01 → 2022-12-31': ('2020-2022', 3),
    
    # AJOUT : Format Court (Eboulements) -> On pointe vers la même durée,
    '2001-2008': ('2001-2008', 8),
    '2009-2012': ('2009-2012', 4),
    '2013-2015': ('2013-2015', 3),
    '2016-2019': ('2016-2019', 4),
    '2020-2022': ('2020-2022', 3)
}

def process_periods(df):
    """Trouve automatiquement la colonne de période et applique le mapping"""
    period_col = None
    for col in df.columns:
        if 'period' in col.lower() or 'période' in col.lower():
            period_col = col
            break
    
    if period_col is None:
        print(f"Attention: Aucune colonne période trouvée dans {df.shape}")
        return df
    
    # Appliquer le mapping
    df['periode_clean'] = df[period_col].map(lambda x: period_map.get(str(x).strip(), (str(x), 1))[0])
    df['duree'] = df[period_col].map(lambda x: period_map.get(str(x).strip(), (str(x), 1))[1])
    
    return df

df_eboulements = process_periods(df_eboulements)
df_meteo = process_periods(df_meteo)
df_mer = process_periods(df_mer)
df_marnage = process_periods(df_marnage)

# --- 3. PRÉPARATION DE LA VÉRITÉ TERRAIN (ÉBOULEMENTS) ---
# Normaliser par la durée pour avoir une fréquence annuelle
nb_col = 'nb_éboulements' if 'nb_éboulements' in df_eboulements.columns else 'nb_eboulements'
df_eboulements['eboulements_par_an'] = df_eboulements[nb_col] / df_eboulements['duree']

# NORMALISATION UNIQUE DES ÉBOULEMENTS (pour tous les graphiques)
scaler_eboulements = MinMaxScaler()
df_eboulements['eboulements_norm'] = scaler_eboulements.fit_transform(
    df_eboulements[['eboulements_par_an']]
)

target_df = df_eboulements[['periode_clean', 'eboulements_par_an', 'eboulements_norm']].copy()

print(f" Données chargées:")
print(f"   - Éboulements: {len(df_eboulements)} périodes")
print(f"   - Météo: {df_meteo.shape[1]-3} indicateurs sur {len(df_meteo)} périodes")
print(f"   - Mer: {df_mer.shape[1]-3} indicateurs sur {len(df_mer)} périodes")
print(f"   - Marnage: {df_marnage.shape[1]-3} indicateurs sur {len(df_marnage)} périodes")

# Vérification de la normalisation des éboulements
print(f" Vérification normalisation éboulements:")
for _, row in target_df.iterrows():
    print(f"   {row['periode_clean']}: {row['eboulements_par_an']:.2f} → normalisé: {row['eboulements_norm']:.3f}")

# --- 4. MOTEUR D'ANALYSE AUTOMATIQUE UNIVERSEL ---

def analyze_all_indicators(source_df, source_name, target_df, verbose=True):
    """
    Analyse AUTOMATIQUEMENT tous les indicateurs numériques d'un DataFrame.
    
    Détecte automatiquement toutes les colonnes numériques
    Applique la normalisation intelligente (cumul/an vs intensité)
    Génère les graphiques de corrélation
    
    Pas besoin de lister les indicateurs manuellement !
    """
    charts = []
    scaler = MinMaxScaler()
    
    # Colonnes à ignorer automatiquement (métadonnées, pas des indicateurs)
    ignore_patterns = ['period', 'période', 'duree', 'durée', 'nb_points', 'jours_disponibles', 
                       'points', 'disponible', 'clean', 'annee', 'année']
    
    # Identifier TOUTES les colonnes numériques
    numeric_cols = source_df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Filtrer les colonnes métadonnées
    indicator_cols = [col for col in numeric_cols 
                     if not any(pattern in col.lower() for pattern in ignore_patterns)]
    
    if verbose:
        print(f" {source_name}: {len(indicator_cols)} indicateurs détectés")
        print(f"   Colonnes analysées: {', '.join(indicator_cols[:5])}{'...' if len(indicator_cols) > 5 else ''}")
    
    for col in indicator_cols:
        # --- LOGIQUE INTELLIGENTE DE NORMALISATION ---
        # Détecter si c'est un CUMUL (à diviser par durée) ou une INTENSITÉ (garder tel quel)
        
        keywords_cumulative = ['cum', 'total', 'sum', 'somme', 'nb_', 'nombre', 'jours_']
        keywords_intensity = ['moyenne', 'max', 'min', 'std', 'écart', 
                             'ratio', 'pct', 'pourcentage', 'degré', 
                             'hpa', 'hauteur', 'vitesse']
        
        col_lower = col.lower()
        is_cumulative = any(k in col_lower for k in keywords_cumulative)
        
        # Exception: si c'est une intensité, ce n'est pas un cumul
        if any(k in col_lower for k in keywords_intensity):
            is_cumulative = False
        
        # Préparer les données
        temp_df = source_df[['periode_clean', 'duree', col]].copy()
        
        # Appliquer la normalisation temporelle si nécessaire
        if is_cumulative:
            temp_df[col] = temp_df[col] / temp_df['duree']
            col_label = f"{col} (par an)"
        else:
            col_label = f"{col} (intensité)"
        
        # Fusionner avec les éboulements
        merged = pd.merge(target_df, temp_df, on='periode_clean', how='inner')
        
        if merged.empty or merged[col].isna().all():
            continue
        
        # NORMALISATION CORRECTE : Normaliser UNIQUEMENT l'indicateur
        # Les éboulements sont déjà normalisés (eboulements_norm)
        scaler_indicator = MinMaxScaler()
        merged['indicateur_norm'] = scaler_indicator.fit_transform(merged[[col]])
        
        # Format long pour Altair
        df_long = merged[['periode_clean', 'eboulements_norm', 'indicateur_norm']].melt(
            'periode_clean', 
            var_name='Type', 
            value_name='Valeur'
        )
        
        # Renommer pour la légende
        df_long['Type'] = df_long['Type'].replace({
            'eboulements_norm': 'Éboulements/an',
            'indicateur_norm': col_label
        })
        
        # Création du graphique
        chart = alt.Chart(df_long).mark_line(point=True).encode(
            x=alt.X('periode_clean:N', title='', axis=alt.Axis(labelAngle=-45)),
            y=alt.Y('Valeur:Q', title='Valeur normalisée (0-1)', scale=alt.Scale(domain=[0, 1])),
            color=alt.Color('Type:N', legend=alt.Legend(orient='top', title=None)),
            tooltip=['periode_clean', 'Type', alt.Tooltip('Valeur:Q', format='.3f')]
        ).properties(
            title=f"{col_label}",
            width=280,
            height=180
        )
        
        charts.append(chart)
    
    return charts

# --- 5. EXÉCUTION AUTOMATIQUE ---

print("\n" + "="*60)
print("ANALYSE AUTOMATIQUE DE TOUS LES INDICATEURS")
print("="*60)

# Analyser automatiquement TOUS les indicateurs météo
charts_meteo = analyze_all_indicators(df_meteo, "Météo", target_df)

# Analyser automatiquement TOUS les indicateurs mer
charts_mer = analyze_all_indicators(df_mer, "Mer", target_df)

# Analyser automatiquement TOUS les indicateurs marnage
charts_marnage = analyze_all_indicators(df_marnage, "Marnage", target_df)

# Combiner tous les graphiques
all_charts = charts_meteo + charts_mer + charts_marnage

print(f" Génération terminée: {len(all_charts)} graphiques créés")
print("="*60)

# --- 6. SAUVEGARDE EN IMAGE ---
if all_charts:
    # Grille de 3 colonnes pour une meilleure lisibilité
    grid = alt.vconcat(*[
        alt.hconcat(*all_charts[i:i+3]) 
        for i in range(0, len(all_charts), 3)
    ]).resolve_scale(color='independent')
    
    display(grid)
    # Sauvegarder dans un fichier image
    output_file = "correlation_graphs.png"
    grid.save(output_file, scale_factor=2.0)
    print(f" Graphiques sauvegardés dans: {output_file}")
    print(f"   {len(all_charts)} graphiques au total")
else:
    print(" Aucun graphique généré. Vérifiez vos données.")


 Données chargées:
   - Éboulements: 5 périodes
   - Météo: 43 indicateurs sur 5 périodes
   - Mer: 14 indicateurs sur 5 périodes
   - Marnage: 9 indicateurs sur 5 périodes
 Vérification normalisation éboulements:
   2001-2008: 21.50 → normalisé: 0.000
   2009-2012: 33.50 → normalisé: 0.986
   2013-2015: 28.00 → normalisé: 0.534
   2016-2019: 27.25 → normalisé: 0.473
   2020-2022: 33.67 → normalisé: 1.000

ANALYSE AUTOMATIQUE DE TOUS LES INDICATEURS
 Météo: 43 indicateurs détectés
   Colonnes analysées: Unnamed: 0, nb_jours, pluie_cum_mm, jours_pluie, jours_forte_pluie...
 Mer: 13 indicateurs détectés
   Colonnes analysées: hs_moy, hs_max, hs_mediane, t02_moy, t02_max...
 Mer: 13 indicateurs détectés
   Colonnes analysées: hs_moy, hs_max, hs_mediane, t02_moy, t02_max...
 Marnage: 7 indicateurs détectés
   Colonnes analysées: marnage_moy_m, marnage_max_m, marnage_min_m, marnage_std_m, jours_marnage>8m...
 Génération terminée: 62 graphiques créés
 Marnage: 7 indicateurs détectés
   Colon

alt.VConcatChart(...)

 Graphiques sauvegardés dans: correlation_graphs.png
   62 graphiques au total


In [46]:
# --- 7. CALCUL DES CORRÉLATIONS ET CLASSEMENT ---
from scipy.stats import pearsonr, spearmanr

def calculate_correlations(source_df, source_name, target_df):
    """
    Calcule la corrélation entre chaque indicateur et les éboulements.
    Retourne un DataFrame trié par corrélation absolue décroissante.
    """
    results = []
    
    # Colonnes à ignorer
    ignore_patterns = ['period', 'période', 'duree', 'durée', 'nb_points', 'jours_disponibles', 
                       'points', 'disponible', 'clean', 'annee', 'année']
    
    # Identifier les colonnes numériques
    numeric_cols = source_df.select_dtypes(include=[np.number]).columns.tolist()
    indicator_cols = [col for col in numeric_cols 
                     if not any(pattern in col.lower() for pattern in ignore_patterns)]
    
    for col in indicator_cols:
        # Normalisation temporelle intelligente
        keywords_cumulative = ['cum', 'total', 'sum', 'somme', 'nb_', 'nombre', 'jours_']
        keywords_intensity = ['moy', 'moyenne', 'max', 'min', 'std', 'écart', 
                             'ratio', 'pct', 'pourcentage', 'degré', 
                             'hpa', 'pression', 'hauteur', 'vitesse']
        
        col_lower = col.lower()
        is_cumulative = any(k in col_lower for k in keywords_cumulative)
        if any(k in col_lower for k in keywords_intensity):
            is_cumulative = False
        
        # Préparer les données
        temp_df = source_df[['periode_clean', 'duree', col]].copy()
        
        if is_cumulative:
            temp_df[col] = temp_df[col] / temp_df['duree']
            col_label = f"{col} (par an)"
        else:
            col_label = f"{col}"
        
        # Fusionner avec les éboulements
        merged = pd.merge(target_df, temp_df, on='periode_clean', how='inner')
        
        if merged.empty or merged[col].isna().any() or len(merged) < 3:
            continue
        
        # Calculer les corrélations
        try:
            pearson_corr, pearson_pval = pearsonr(merged['eboulements_par_an'], merged[col])
            spearman_corr, spearman_pval = spearmanr(merged['eboulements_par_an'], merged[col])
            
            results.append({
                'Source': source_name,
                'Indicateur': col,
                'Type': 'Par an' if is_cumulative else 'Intensité',
                'Pearson_r': pearson_corr,
                'Pearson_pval': pearson_pval,
                'Spearman_r': spearman_corr,
                'Spearman_pval': spearman_pval,
                'Pearson_abs': abs(pearson_corr),
                'Spearman_abs': abs(spearman_corr)
            })
        except Exception as e:
            continue
    
    return pd.DataFrame(results)

print("\n" + "="*60)
print("CALCUL DES CORRÉLATIONS")
print("="*60)

# Calculer pour Météo et Mer
corr_meteo = calculate_correlations(df_meteo, "Météo", target_df)
corr_mer = calculate_correlations(df_mer, "Mer", target_df)
corr_marnage = calculate_correlations(df_marnage, "Marnage", target_df)

# Combiner et trier
corr_all = pd.concat([corr_meteo, corr_mer, corr_marnage], ignore_index=True)
corr_all = corr_all.sort_values('Pearson_abs', ascending=False)

# ---------------------------------------------------------
# TOP 10 CORRÉLATIONS POSITIVES 
# ---------------------------------------------------------
print(f"\nTOP 10 CORRÉLATIONS POSITIVES (r > 0)")
print("-" * 80)
# On filtre les r > 0 et on trie du plus grand au plus petit
pos_corr = corr_all[corr_all['Pearson_r'] > 0].sort_values('Pearson_r', ascending=False).head(10)

if pos_corr.empty:
    print("Aucune corrélation positive significative trouvée.")
else:
    for idx, row in pos_corr.iterrows():
        # Définition des étoiles de significativité
        seuil = "***" if row['Pearson_pval'] < 0.01 else "**" if row['Pearson_pval'] < 0.05 else "*" if row['Pearson_pval'] < 0.1 else ""
        print(f"{idx+1:2d}. {row['Source']:6s} | {row['Indicateur']:40s} | r={row['Pearson_r']:+.3f} {seuil:3s} (p={row['Pearson_pval']:.3f})")

# ---------------------------------------------------------
# TOP 10 CORRÉLATIONS NÉGATIVES 
# ---------------------------------------------------------
print(f"\nTOP 10 CORRÉLATIONS NÉGATIVES (r < 0)")
print("-" * 80)
# On filtre les r < 0 et on trie du plus petit au plus grand (ex: -0.9 est plus fort que -0.1)
neg_corr = corr_all[corr_all['Pearson_r'] < 0].sort_values('Pearson_r', ascending=True).head(10)

if neg_corr.empty:
    print("Aucune corrélation négative significative trouvée.")
else:
    # On réinitialise l'index pour l'affichage (1 à 10)
    for i, (idx, row) in enumerate(neg_corr.iterrows()):
        seuil = "***" if row['Pearson_pval'] < 0.01 else "**" if row['Pearson_pval'] < 0.05 else "*" if row['Pearson_pval'] < 0.1 else ""
        print(f"{i+1:2d}. {row['Source']:6s} | {row['Indicateur']:40s} | r={row['Pearson_r']:+.3f} {seuil:3s} (p={row['Pearson_pval']:.3f})")



CALCUL DES CORRÉLATIONS

TOP 10 CORRÉLATIONS POSITIVES (r > 0)
--------------------------------------------------------------------------------
20. Météo  | jours_vent_fort_60                       | r=+0.809 *   (p=0.097)
33. Météo  | nb_seq_depression_3j                     | r=+0.772     (p=0.126)
25. Météo  | energie_vent_cumulee                     | r=+0.729     (p=0.162)
 8. Météo  | nb_seq_seche_10j                         | r=+0.721     (p=0.169)
34. Météo  | nb_jours_depression_instantanée          | r=+0.717     (p=0.173)
26. Météo  | nb_jours_vent_dir_Ouest                  | r=+0.716     (p=0.173)
46. Mer    | t02_moy                                  | r=+0.678     (p=0.208)
42. Météo  | nb_combinaisons_critiques                | r=+0.671     (p=0.215)
57. Marnage | marnage_max_m                            | r=+0.614     (p=0.270)
39. Météo  | jours_pluie_et_vent_fort                 | r=+0.605     (p=0.279)

TOP 10 CORRÉLATIONS NÉGATIVES (r < 0)
-------------------------

In [47]:
# --- 8. GÉNÉRATION DES GRAPHIQUES (TOP 5 POSITIF & TOP 5 NEGATIF) ---
from sklearn.preprocessing import MinMaxScaler
import altair as alt

def generate_charts_from_list(corr_subset, source_dfs, target_df):
    """
    Génère une liste de graphiques Altair à partir d'un DataFrame de corrélations filtré.
    """
    charts = []
    
    for _, row in corr_subset.iterrows():
        source_name = row['Source']
        col = row['Indicateur']
        is_cumulative = row['Type'] == 'Par an'
        r_value = row['Pearson_r']
        
        # Sélectionner le bon DataFrame source
        # Note: Assurez-vous que les clés correspondent aux noms dans votre colonne 'Source'
        source_df = source_dfs.get(source_name)
        
        if source_df is None:
            print(f"Source non trouvée: {source_name}")
            continue
        
        # Préparer les données
        temp_df = source_df[['periode_clean', 'duree', col]].copy()
        
        if is_cumulative:
            temp_df[col] = temp_df[col] / temp_df['duree']
            col_label = f"{col} (par an)"
        else:
            col_label = f"{col}"
        
        # Fusionner avec les éboulements
        merged = pd.merge(target_df, temp_df, on='periode_clean', how='inner')
        
        if merged.empty or merged[col].isna().all():
            continue
        
        # Normalisation (0-1) pour comparaison visuelle
        scaler = MinMaxScaler()
        # On normalise les deux colonnes pour qu'elles se superposent bien
        merged['eboulements_norm'] = scaler.fit_transform(merged[['eboulements_par_an']])
        merged['indicateur_norm'] = scaler.fit_transform(merged[[col]])
        
        # Format long pour Altair
        df_long = merged[['periode_clean', 'eboulements_norm', 'indicateur_norm']].melt(
            'periode_clean', 
            var_name='Type', 
            value_name='Valeur'
        )
        
        # Renommage pour la légende
        df_long['Type'] = df_long['Type'].replace({
            'eboulements_norm': 'Éboulements (Ref)',
            'indicateur_norm': col_label
        })
        
        # Titre dynamique avec flèche pour indiquer la direction
        title_text = f" {col_label} | r={r_value:+.2f}"
        
        # Création du graphique
        chart = alt.Chart(df_long).mark_line(point=True).encode(
            x=alt.X('periode_clean:N', title='', axis=alt.Axis(labelAngle=-45)),
            y=alt.Y('Valeur:Q', title=None, axis=None), # On cache l'axe Y pour épurer (car normalisé)
            color=alt.Color('Type:N', legend=alt.Legend(orient='top', title=None)),
            tooltip=['periode_clean', 'Type', alt.Tooltip('Valeur:Q', format='.2f')]
        ).properties(
            title=title_text,
            width=280,
            height=180
        )
        
        charts.append(chart)
    
    return charts

print("\n" + "="*60)
print("GÉNÉRATION DES GRAPHIQUES (POS VS NEG)")
print("="*60)

# 1. Sélectionner les 5 meilleurs POSITIFS
top_5_pos = corr_all[corr_all['Pearson_r'] > 0].sort_values('Pearson_r', ascending=False).head(5)

# 2. Sélectionner les 5 meilleurs NÉGATIFS (les plus proches de -1)
top_5_neg = corr_all[corr_all['Pearson_r'] < 0].sort_values('Pearson_r', ascending=True).head(5)

# 3. Combiner les deux listes pour le traitement
target_selection = pd.concat([top_5_pos, top_5_neg])

# Dictionnaire des sources de données
sources_dict = {'Météo': df_meteo, 'Mer': df_mer, 'Marnage': df_marnage} 

# Génération
charts_list = generate_charts_from_list(target_selection, sources_dict, target_df)

print(f"{len(charts_list)} graphiques générés (5 Positifs + 5 Négatifs)")

# Affichage et sauvegarde
if charts_list:
    # On arrange en grille de 2 colonnes pour bien voir (ou 3 selon préférence)
    grid = alt.vconcat(*[
        alt.hconcat(*charts_list[i:i+5]) 
        for i in range(0, len(charts_list), 5)
    ]).resolve_scale(color='independent')
    
    display(grid)
    
    # Sauvegarder
    output_file = "top_pos_neg_correlations.png"
    try:
        grid.save(output_file, scale_factor=2.0)
        print(f"Graphiques sauvegardés dans: {output_file}")
    except Exception as e:
        print(f"Erreur sauvegarde: {e}")
else:
    print("Aucun graphique généré (vérifiez les données)")

print("="*60)


GÉNÉRATION DES GRAPHIQUES (POS VS NEG)
10 graphiques générés (5 Positifs + 5 Négatifs)
10 graphiques générés (5 Positifs + 5 Négatifs)


alt.VConcatChart(...)

Graphiques sauvegardés dans: top_pos_neg_correlations.png
